In [21]:
import json
from pandas.io.json import json_normalize

from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kurtosis, skew, norm, entropy, gaussian_kde
from sklearn.preprocessing import StandardScaler
#from imblearn.over_sampling import SMOTE
import xgboost as xgb
import os
from bokeh.plotting import figure, show, output_notebook
from bokeh.palettes import Spectral11, colorblind, Inferno, BuGn, brewer
from bokeh.models import HoverTool, value, LabelSet, Legend, ColumnDataSource,LinearColorMapper,BasicTicker, PrintfTickFormatter, ColorBar
import bokeh.plotting as bk

from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xg
from sklearn.model_selection import GridSearchCV
from datetime import date
import calendar
from pandas.tseries.holiday import USFederalHolidayCalendar
import datetime
from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler

from fancyimpute import BiScaler, KNN, NuclearNormMinimization, SoftImpute, MICE
bk.output_notebook()
plt.style.use('fivethirtyeight')
%matplotlib inline

ImportError: No module named xgboost

In [3]:
csvs=os.listdir('data')
csvs

['test_V2.csv', 'sample_submission_V2.csv', 'train_V2.csv']

In [5]:
path='data/'
testdf=pd.read_csv(path+csvs[0])
sampdf=pd.read_csv(path+csvs[1])
traindf=pd.read_csv(path+csvs[2])

In [8]:
shared_cols=[x for x in list(testdf) if x in list(traindf)]
mergeddf=pd.concat([traindf[shared_cols],testdf])

In [9]:
traindf.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,...,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,...,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,...,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,...,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,...,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [10]:
sampdf.head()

,Id,winPlacePerc
0,9329eb41e215eb,1
1,639bd0dcd7bda8,1
2,63d5c8ef8dfe91,1
3,cf5b81422591d1,1
4,ee6a295187ba21,1


In [14]:
edacount=0
labels=['train','test','merged']
for i in [traindf,testdf,mergeddf]:
    print labels[edacount]+' EDA'
    print '\nSHAPE'
    print i.shape
    print '\nINFO'
    print i.info()
    print '\nNULL COUNT'
    print i.isnull().sum()
    print '\nNUMBER OF UNIQUE VALUES'
    print i.nunique()
    edacount+=1
    print '#####################################################################\n\n'

train EDA

SHAPE
(4446966, 29)

INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4446966 entries, 0 to 4446965
Data columns (total 29 columns):
Id                 object
groupId            object
matchId            object
assists            int64
boosts             int64
damageDealt        float64
DBNOs              int64
headshotKills      int64
heals              int64
killPlace          int64
killPoints         int64
kills              int64
killStreaks        int64
longestKill        float64
matchDuration      int64
matchType          object
maxPlace           int64
numGroups          int64
rankPoints         int64
revives            int64
rideDistance       float64
roadKills          int64
swimDistance       float64
teamKills          int64
vehicleDestroys    int64
walkDistance       float64
weaponsAcquired    int64
winPoints          int64
winPlacePerc       float64
dtypes: float64(6), int64(19), object(4)
memory usage: 983.9+ MB
None

NULL COUNT
Id                 0
groupI

# Drop Columns that Only Have One Value

In [15]:
# remove useless columns 
unique_cols=mergeddf.nunique().sort_values()
mergeddf=mergeddf[[x for x in set(list(mergeddf)) if x not in set(unique_cols[unique_cols==1].index)]]

# Categorical features
## Look for numeric values that behave as if categorical -- scratched for first round of predictions

In [16]:
# look for numerical values that act as cat
actual_cat=[]
actual_int=[]
merged_ints=mergeddf[mergeddf.dtypes[(mergeddf.dtypes!='object')&(mergeddf.dtypes!='bool')].index]

for col in list(merged_ints):
    if merged_ints[col].nunique()*1.0/merged_ints.shape[0] <= 0.2: # if the number of unique values is less than 20%
        actual_cat.append(col)
    else:
        actual_int.append(col)

print 'actually_categorical'
print actual_cat
print 'actually numeric'
print actual_int

actually_categorical
['walkDistance', 'weaponsAcquired', 'numGroups', 'matchDuration', 'rankPoints', 'winPoints', 'heals', 'boosts', 'DBNOs', 'swimDistance', 'revives', 'vehicleDestroys', 'rideDistance', 'maxPlace', 'longestKill', 'kills', 'teamKills', 'killPoints', 'assists', 'headshotKills', 'killPlace', 'damageDealt', 'roadKills', 'killStreaks']
actually numeric
[]


In [17]:
sns.set(font_scale=1.25)

print merged_df[actual_cat].describe()
merged_df[actual_cat].hist(edgecolor='black', linewidth=1.2)
fig=plt.gcf()
fig.set_size_inches(15,10)
plt.suptitle("Distribution of Numeric Features")
plt.show()

NameError: name 'sns' is not defined